In [ ]:
from azureml.core import Workspace, Experiment,Dataset

ws = Workspace.get(name="quick-starts-ws-121452")
exp = Experiment(workspace=ws, name="hyperdrive-test")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

In [ ]:
run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

cluster_name = 'hd-cluster'

try:
    compute_target = ComputeTarget(ws, cluster_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', min_nodes=1, max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

In [ ]:
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, ct.type, ct.provisioning_state)

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,choice
import os

In [ ]:
# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C': choice(0.01, 0.1, 1, 10, 100, 1000), 
        '--max_iter': choice(25, 50, 100, 200)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval = 3, slack_factor = 0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = './', 
              entry_script = 'train.py', 
              compute_target = compute_target)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling = ps, 
                                     primary_metric_name='Accuracy', 
                                     primary_metric_goal= PrimaryMetricGoal.MAXIMIZE, 
                                     policy = policy, 
                                     max_total_runs = 25, 
                                     max_concurrent_runs = 4, 
                                     estimator = est)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(config = hyperdrive_config)

In [ ]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

In [ ]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
#parameter_values = best_run.get_details()['runDefinition']['Arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])

best_run

In [ ]:
best_run.get_file_names()

In [ ]:
model = best_run.register_model(model_name='model_hd', model_path='outputs/model.joblib')

## Using AutoML

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

url_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
data = TabularDatasetFactory.from_delimited_files(path=url_path)

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(data)

In [ ]:
y_df = pd.DataFrame(y,columns=['y'])
y_df.head()

In [ ]:
total_data = pd.concat([x,y_df],axis=1)
total_data.head()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_data,test_data = train_test_split(total_data,test_size = 0.3,random_state = 42,shuffle=True)

In [ ]:
train_data.to_csv('training/training_data.csv')

In [ ]:
exp = Experiment(workspace=ws, name="AutoML-test")

In [ ]:
datastore = ws.get_default_datastore()
datastore

In [ ]:
datastore.upload(src_dir='training/',target_path='data/')

In [ ]:
train_dataset = Dataset.Tabular.from_delimited_files(path=[(datastore,('data/training_data.csv'))])

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_dataset,
    label_column_name='y',
    n_cross_validations=5,
    compute_target=compute_target, 
    iterations=50,
    max_concurrent_iterations=8,
    max_cores_per_iteration=-1)

In [ ]:
# Submit your automl run

### YOUR CODE HERE ###

from azureml.widgets import RunDetails
automl_run = exp.submit(automl_config, show_output=True)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###

best_automl_run, best_model = automl_run.get_output()

In [ ]:
best_automl_run.register_model(model_name = "automl.pkl", model_path = './outputs/')
print(best_model._final_estimator)

In [ ]:
compute_target.delete()